# Boletín de pobreza feb 2024
ivan corredor 20-02-2024

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
coding: "latin-1"

In [9]:
#pd.options.display.max_info_rows= 50
pd.options.display.float_format = '{:.2f}'.format 
pd.options.display.max_rows = None
pd.options.display.max_columns = None
#pd.options.display.max_colwidth = 300

In [4]:
#ruta entrada
p1 = "C:/Users/ivan.corredor/visor_pobreza/"
#ruta salida pobreza mon
p2 = "C:/Users/ivan.corredor/pobreza_calculos_2019_2022/2023/pobreza monetaria/salidas/"
#ruta salida ipm
p3 = "C:/Users/ivan.corredor/pobreza_calculos_2019_2022/2023/multidimensional/salidas/"

In [5]:
#cargue base 2022
df_vict = pd.read_csv(p1 + "vic_4_periodos.csv", sep = ";", decimal=",")

In [6]:
df_vict.tail()

,directorio,secuencia_p,orden,mes,es_victima,es_desplazado,decada_vic,fex_c_viv,nper,li,lp,pobre,indigente,npobres,nindigentes,clase,dominio,sexo,edad,ingtot,fex_c,depto,fex_dpto,no_victima,periodo,dpto
3081224,7309047.00,1.00,3.00,12,NaN,NaN,NaN,NaN,3.00,229304.31,499129.47,0.00,0.00,0.00,0.00,1,PEREIRA,1.00,47.00,500000.00,17.04,66,17.04,1,2022,66.00
3081225,7309048.00,1.00,1.00,12,NaN,NaN,NaN,NaN,4.00,229304.31,499129.47,0.00,0.00,0.00,0.00,1,PEREIRA,2.00,45.00,4000000.00,30.47,66,30.47,1,2022,66.00
3081226,7309048.00,1.00,2.00,12,NaN,NaN,NaN,NaN,4.00,229304.31,499129.47,0.00,0.00,0.00,0.00,1,PEREIRA,1.00,54.00,3000000.00,30.47,66,30.47,1,2022,66.00
3081227,7309048.00,1.00,3.00,12,NaN,NaN,NaN,NaN,4.00,229304.31,499129.47,0.00,0.00,0.00,0.00,1,PEREIRA,1.00,28.00,2000000.00,30.47,66,30.47,1,2022,66.00
3081228,7309048.00,1.00,4.00,12,NaN,NaN,NaN,NaN,4.00,229304.31,499129.47,0.00,0.00,0.00,0.00,1,PEREIRA,2.00,14.00,NaN,30.47,66,30.47,1,2022,66.00


## Pobreza monetaria

### Sexo

In [7]:
### agrupar los datos para sacar tabla y gráfica
sexo_h = df_vict.groupby(["periodo","sexo"]).apply(
    lambda x: pd.Series({
        # vic tot ipm
        '%_vic_hechos': round((x['pobre'] * x['es_victima']* x['fex_c_viv']).sum() / (x['es_victima'] * x['fex_c_viv']).sum() * 100, 1),
        'vic_hechos': round((x['pobre'] * x['fex_c_viv']).sum(), 0)})
).reset_index()

sexo_d = df_vict.groupby(["periodo","sexo"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_des': round((x['pobre'] * x['es_desplazado'] * x['fex_c_viv']).sum() / (x['es_desplazado'] * x['fex_c_viv']).sum() * 100, 1),
        'des': round((x['pobre'] * x['es_desplazado'] *  x['fex_c_viv']).sum(), 0)
    })
).reset_index()


sexo_nv = df_vict.groupby(["periodo","sexo"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_nv': round((x['pobre'] * x["no_victima"] * x['fex_c']).sum() / (x["no_victima"] * x["fex_c"]).sum() * 100, 1),
        'no_vic': round((x['pobre'] * x["no_victima"]* x["fex_c"]).sum(), 0)
    })
).reset_index()


sexo = pd.merge(sexo_h, sexo_d, on =["periodo","sexo"])
sexo1 = pd.merge(sexo, sexo_nv, on =["periodo","sexo"])
sexo1

,periodo,sexo,%_vic_hechos,vic_hechos,%_des,des,%_nv,no_vic
0,2019,1.00,41.80,1525190.00,42.90,1447807.00,34.30,7581218.00
1,2019,2.00,43.70,1846439.00,44.60,1716410.00,35.70,8012825.00
2,2020,1.00,50.10,1822038.00,51.40,1746531.00,40.60,8841158.00
3,2020,2.00,53.50,2304785.00,55.10,2184427.00,41.90,9194075.00
4,2021,1.00,48.00,1761021.00,49.40,1694971.00,37.10,8155305.00
5,2021,2.00,51.80,2256079.00,53.40,2154396.00,38.70,8568398.00
6,2022,1.00,48.50,1819514.00,49.70,1737191.00,33.90,7297262.00
7,2022,2.00,51.50,2298522.00,53.20,2180515.00,35.40,7879266.00


### Rango etario

In [8]:
def rang_ed(df):
    if df["edad"]<=5:
        x = "Entre 0 y 5 años"
    elif (df["edad"]>5 and df["edad"]<=11):
        x = "Entre 6 y 11 años"
    elif (df["edad"]>11 and df["edad"]<=17):
        x = "Entre 12 y 17 años"
    elif (df["edad"]>17 and df["edad"]<=28):
        x = "Entre 18 y 28 años"
    elif (df["edad"]>28 and df["edad"]<=59):
        x = "Entre 29 y 59 años"
    else:
        x = "60 años o más"
    return x
    
df_vict["grupo_etario"] = df_vict.apply(rang_ed, axis = 1)

In [9]:
### agrupar los datos para sacar tabla y gráfica
ge_h = df_vict.groupby(["periodo","grupo_etario"]).apply(
    lambda x: pd.Series({
        # vic tot ipm
        '%_vic_hechos': round((x['pobre'] * x['es_victima']* x['fex_c_viv']).sum() / (x['es_victima'] * x['fex_c_viv']).sum() * 100, 1),
        'vic_hechos': round((x['pobre'] * x['fex_c_viv']).sum(), 0)})
).reset_index()

ge_d = df_vict.groupby(["periodo","grupo_etario"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_des': round((x['pobre'] * x['es_desplazado'] * x['fex_c_viv']).sum() / (x['es_desplazado'] * x['fex_c_viv']).sum() * 100, 1),
        'des': round((x['pobre'] * x['es_desplazado'] *  x['fex_c_viv']).sum(), 0)
    })
).reset_index()


ge_nv = df_vict.groupby(["periodo","grupo_etario"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_nv': round((x['pobre'] * x["no_victima"] * x['fex_c']).sum() / (x["no_victima"] * x["fex_c"]).sum() * 100, 1),
        'no_vic': round((x['pobre'] * x["no_victima"]* x["fex_c"]).sum(), 0)
    })
).reset_index()


ge = pd.merge(ge_h, ge_d, on =["periodo","grupo_etario"])
ge1 = pd.merge(ge, ge_nv, on =["periodo","grupo_etario"])
ge1

,periodo,grupo_etario,%_vic_hechos,vic_hechos,%_des,des,%_nv,no_vic
0,2019,60 años o más,41.20,357612.00,44.60,311352.00,22.70,1304079.00
1,2019,Entre 0 y 5 años,54.00,251100.00,53.80,246223.00,50.50,2291755.00
2,2019,Entre 12 y 17 años,52.70,307515.00,52.60,300864.00,48.40,2211811.00
3,2019,Entre 18 y 28 años,38.20,587128.00,38.50,567115.00,32.40,2654109.00
4,2019,Entre 29 y 59 años,39.70,1473395.00,40.80,1350106.00,28.30,4720456.00
5,2019,Entre 6 y 11 años,55.50,394880.00,55.20,388555.00,50.10,2411832.00
6,2020,60 años o más,41.10,336607.00,43.70,287935.00,27.10,1593349.00
7,2020,Entre 0 y 5 años,61.80,244975.00,61.80,241945.00,56.30,2549112.00
8,2020,Entre 12 y 17 años,63.60,568972.00,63.90,565405.00,52.70,2199582.00
9,2020,Entre 18 y 28 años,49.40,880723.00,50.10,863086.00,40.30,3184720.00


### Territorial

#### Zona geográfica

In [10]:
### agrupar los datos para sacar tabla y gráfica
clase_h = df_vict.groupby(["periodo","clase"]).apply(
    lambda x: pd.Series({
        # vic tot ipm
        '%_vic_hechos': round((x['pobre'] * x['es_victima']* x['fex_c_viv']).sum() / (x['es_victima'] * x['fex_c_viv']).sum() * 100, 1),
        'vic_hechos': round((x['pobre'] * x['fex_c_viv']).sum(), 0)})
).reset_index()

clase_d = df_vict.groupby(["periodo","clase"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_des': round((x['pobre'] * x['es_desplazado'] * x['fex_c_viv']).sum() / (x['es_desplazado'] * x['fex_c_viv']).sum() * 100, 1),
        'des': round((x['pobre'] * x['es_desplazado'] *  x['fex_c_viv']).sum(), 0)
    })
).reset_index()


clase_nv = df_vict.groupby(["periodo","clase"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_nv': round((x['pobre'] * x["no_victima"] * x['fex_c']).sum() / (x["no_victima"] * x["fex_c"]).sum() * 100, 1),
        'no_vic': round((x['pobre'] * x["no_victima"]* x["fex_c"]).sum(), 0)
    })
).reset_index()


clase = pd.merge(clase_h, clase_d, on =["periodo","clase"])
clase1 = pd.merge(clase, clase_nv, on =["periodo","clase"])
clase1

,periodo,clase,%_vic_hechos,vic_hechos,%_des,des,%_nv,no_vic
0,2019,1,38.70,2096161.00,39.90,1964611.00,31.70,11075997.00
1,2019,2,51.90,1275468.00,52.40,1199606.00,46.90,4518046.00
2,2020,1,55.30,2883186.00,57.30,2745221.00,41.00,14187964.00
3,2020,2,45.60,1243637.00,46.10,1185737.00,42.30,3847269.00
4,2021,1,51.20,2640535.00,53.30,2530015.00,36.30,12748915.00
5,2021,2,48.10,1376565.00,48.60,1319352.00,43.80,3974788.00
6,2022,1,49.70,2573086.00,51.90,2455401.00,32.00,11050743.00
7,2022,2,50.80,1544949.00,51.20,1462305.00,44.70,4125785.00


#### Departamento

In [11]:
### estandarizar variable dpto
#df_vict["dpto"] = pd.to_numeric(df_vict["dpto"],  errors="coerce").astype('Int8')

In [12]:
#agregar variable nombres de departamentos

df_vict["nombre_dpto"] = df_vict["depto"].replace([5, 8, 11, 13, 15, 17, 18, 19, 20, 23, 25, 27, 
                                                   41, 44, 47, 50, 52, 54, 63, 66, 68, 70, 73, 76,
                                                   81, 85, 86, 88, 91, 94, 95, 97, 99],
                                                   ["Antioquia", "Atlántico", "Bogotá", "Bolívar", "Boyacá",
                                                    "Caldas", "Caquetá", "Cauca", "Cesar", "Córdoba", "Cundinamarca",
                                                    "Chocó", "Huila", "La Guajira", "Magdalena", "Meta", "Nariño",
                                                    "Norte de Santander", "Quindio", "Risaralda", "Santander",
                                                    "Sucre", "Tolima", "Valle del Cauca", "Arauca", "Casanare",
                                                    "Putumayo", "Providencia y Santa Catalina", "Amazonas",
                                                    "Guainía", "Guaviare", "Vaupés", "Vichada"])

In [13]:
#agregar variable código ciudades capítales
df_vict["cod_dane"]= df_vict["dominio"].replace(['ARMENIA','BARRANQUILLA','BOGOTA','BUCARAMANGA','CALI','CARTAGENA',
                                                     'CUCUTA', 'FLORENCIA','IBAGUE','MANIZALES','MEDELLIN','MONTERIA',
                                                     'NEIVA','PASTO','PEREIRA','POPAYAN','QUIBDO','RIOHACHA',
                                                     'SANTA MARTA','SINCELEJO','TUNJA','VALLEDUPAR','VILLAVICENCIO'],
                                                                [63001, 8001, 11001, 68001, 6001, 13001,
                                                                 54001, 18001, 73001, 17001, 5001, 23001,
                                                                 41001, 52001, 66001, 19001, 27001, 44001,
                                                                 47001, 70001, 15001, 20001, 50001])

In [14]:
### agregar ceros a la izquierda para completar el código
df_vict["depto"] = df_vict["depto"].astype(str).str.zfill(2)
### agregar ceros a la izquierda para completar el código
df_vict["cod_dane"] = df_vict["cod_dane"].astype(str).str.zfill(5)

In [15]:
### agrupar los datos para sacar tabla y gráfica
depto_h = df_vict.groupby(["periodo","depto", "nombre_dpto"]).apply(
    lambda x: pd.Series({
        # vic tot ipm
        '%_vic_hechos': round((x['pobre'] * x['es_victima']* x['fex_c_viv']).sum() / (x['es_victima'] * x['fex_c_viv']).sum() * 100, 1),
        'vic_hechos': round((x['pobre'] * x['fex_c_viv']).sum(), 0)})
).reset_index()

depto_d = df_vict.groupby(["periodo","depto", "nombre_dpto"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_des': round((x['pobre'] * x['es_desplazado'] * x['fex_c_viv']).sum() / (x['es_desplazado'] * x['fex_c_viv']).sum() * 100, 1),
        'des': round((x['pobre'] * x['es_desplazado'] *  x['fex_c_viv']).sum(), 0)
    })
).reset_index()


depto_nv = df_vict.groupby(["periodo","depto", "nombre_dpto"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_nv': round((x['pobre'] * x["no_victima"] * x['fex_c']).sum() / (x["no_victima"] * x["fex_c"]).sum() * 100, 1),
        'no_vic': round((x['pobre'] * x["no_victima"]* x["fex_c"]).sum(), 0)
    })
).reset_index()


depto = pd.merge(depto_h, depto_d, on =["periodo","depto", "nombre_dpto"])
depto1 = pd.merge(depto, depto_nv, on =["periodo","depto", "nombre_dpto"])
depto1

,periodo,depto,nombre_dpto,%_vic_hechos,vic_hechos,%_des,des,%_nv,no_vic
0,2019,05,Antioquia,33.30,452351.00,34.90,419783.00,28.10,1482902.00
1,2019,08,Atlántico,26.20,56180.00,26.60,54010.00,27.20,690010.00
2,2019,11,Bogotá,30.40,162862.00,31.30,152009.00,27.10,2156233.00
3,2019,13,Bolívar,54.70,218881.00,55.40,213930.00,45.20,912369.00
4,2019,15,Boyacá,40.70,21825.00,47.00,20978.00,35.00,496862.00
5,2019,17,Caldas,33.80,49382.00,33.80,43118.00,27.50,275417.00
6,2019,18,Caquetá,47.40,126232.00,48.50,121998.00,47.90,247458.00
7,2019,19,Cauca,57.10,190554.00,58.50,168537.00,55.00,703428.00
8,2019,20,Cesar,54.60,165959.00,55.10,158358.00,52.70,562564.00
9,2019,23,Córdoba,54.10,171400.00,54.70,163921.00,50.00,684931.00


#### Ciudades

In [16]:
### agrupar los datos para sacar tabla y gráfica
ciudad_h = df_vict.groupby(["periodo","cod_dane", "dominio"]).apply(
    lambda x: pd.Series({
        # vic tot ipm
        '%_vic_hechos': round((x['pobre'] * x['es_victima']* x['fex_c_viv']).sum() / (x['es_victima'] * x['fex_c_viv']).sum() * 100, 1),
        'vic_hechos': round((x['pobre'] * x['fex_c_viv']).sum(), 0)})
).reset_index()

ciudad_d = df_vict.groupby(["periodo","cod_dane", "dominio"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_des': round((x['pobre'] * x['es_desplazado'] * x['fex_c_viv']).sum() / (x['es_desplazado'] * x['fex_c_viv']).sum() * 100, 1),
        'des': round((x['pobre'] * x['es_desplazado'] *  x['fex_c_viv']).sum(), 0)
    })
).reset_index()


ciudad_nv = df_vict.groupby(["periodo","cod_dane", "dominio"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_nv': round((x['pobre'] * x["no_victima"] * x['fex_c']).sum() / (x["no_victima"] * x["fex_c"]).sum() * 100, 1),
        'no_vic': round((x['pobre'] * x["no_victima"]* x["fex_c"]).sum(), 0)
    })
).reset_index()


ciudad = pd.merge(ciudad_h, ciudad_d, on =["periodo","cod_dane", "dominio"])
ciudad1 = pd.merge(ciudad, ciudad_nv, on =["periodo","cod_dane", "dominio"])
ciudad1

,periodo,cod_dane,dominio,%_vic_hechos,vic_hechos,%_des,des,%_nv,no_vic
0,2019,05001,MEDELLIN,29.50,199989.00,31.20,176770.00,23.90,809835.00
1,2019,06001,CALI,28.40,80464.00,29.70,76234.00,21.50,513589.00
2,2019,08001,BARRANQUILLA,26.20,40342.00,26.80,39174.00,25.50,467090.00
3,2019,11001,BOGOTA,30.40,162862.00,31.30,152009.00,27.10,2156233.00
4,2019,13001,CARTAGENA,40.40,19350.00,41.10,18789.00,34.20,334538.00
5,2019,15001,TUNJA,37.00,2629.00,37.80,2410.00,32.80,62730.00
6,2019,17001,MANIZALES,23.70,7174.00,24.00,6621.00,20.50,83437.00
7,2019,18001,FLORENCIA,47.00,31384.00,47.60,29873.00,43.00,54164.00
8,2019,19001,POPAYAN,51.10,23345.00,53.40,21241.00,44.20,102235.00
9,2019,20001,VALLEDUPAR,38.90,27519.00,39.20,25918.00,40.90,157351.00


## Pobreza extrema

### Sexo

In [17]:
### agrupar los datos para sacar tabla y gráfica
sexo_h_i = df_vict.groupby(["periodo","sexo"]).apply(
    lambda x: pd.Series({
        # vic tot ipm
        '%_vic_hechos': round((x['indigente'] * x['es_victima']* x['fex_c_viv']).sum() / (x['es_victima'] * x['fex_c_viv']).sum() * 100, 1),
        'vic_hechos': round((x['indigente'] * x['fex_c_viv']).sum(), 0)})
).reset_index()

sexo_d_i = df_vict.groupby(["periodo","sexo"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_des': round((x['indigente'] * x['es_desplazado'] * x['fex_c_viv']).sum() / (x['es_desplazado'] * x['fex_c_viv']).sum() * 100, 1),
        'des': round((x['indigente'] * x['es_desplazado'] *  x['fex_c_viv']).sum(), 0)
    })
).reset_index()


sexo_nv_i = df_vict.groupby(["periodo","sexo"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_nv': round((x['indigente'] *  x["no_victima"]*x['fex_c']).sum() / (x["no_victima"] * x["fex_c"]).sum() * 100, 1),
        'no_vic': round((x['indigente'] * x["no_victima"]* x["fex_c"]).sum(), 0)
    })
).reset_index()


sexo_i = pd.merge(sexo_h_i, sexo_d_i, on =["periodo","sexo"])
sexo1_i = pd.merge(sexo_i, sexo_nv_i, on =["periodo","sexo"])
sexo1_i

,periodo,sexo,%_vic_hechos,vic_hechos,%_des,des,%_nv,no_vic
0,2019,1.00,12.70,461816.00,12.90,435649.00,8.90,1976967.00
1,2019,2.00,13.70,579984.00,14.10,543245.00,9.50,2132250.00
2,2020,1.00,19.00,692787.00,19.60,665783.00,14.10,3070604.00
3,2020,2.00,20.50,884041.00,21.20,839803.00,14.90,3258606.00
4,2021,1.00,17.20,630971.00,17.70,608856.00,11.10,2452099.00
5,2021,2.00,18.40,803829.00,19.20,775128.00,11.80,2623827.00
6,2022,1.00,20.40,764977.00,20.90,729985.00,12.40,2667473.00
7,2022,2.00,21.90,977294.00,22.70,928977.00,13.00,2901937.00


### Rango etario

In [18]:
### agrupar los datos para sacar tabla y gráfica
ge_h_i = df_vict.groupby(["periodo","grupo_etario"]).apply(
    lambda x: pd.Series({
        # vic tot ipm
        '%_vic_hechos': round((x['indigente'] * x['es_victima']* x['fex_c_viv']).sum() / (x['es_victima'] * x['fex_c_viv']).sum() * 100, 1),
        'vic_hechos': round((x['indigente'] * x['fex_c_viv']).sum(), 0)})
).reset_index()

ge_d_i = df_vict.groupby(["periodo","grupo_etario"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_des': round((x['indigente'] * x['es_desplazado'] * x['fex_c_viv']).sum() / (x['es_desplazado'] * x['fex_c_viv']).sum() * 100, 1),
        'des': round((x['indigente'] * x['es_desplazado'] *  x['fex_c_viv']).sum(), 0)
    })
).reset_index()


ge_nv_i = df_vict.groupby(["periodo","grupo_etario"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_nv': round((x['indigente'] *  x["no_victima"]*x['fex_c']).sum() / (x["no_victima"] * x["fex_c"]).sum() * 100, 1),
        'no_vic': round((x['indigente'] * x["no_victima"]* x["fex_c"]).sum(), 0)
    })
).reset_index()


ge_i = pd.merge(ge_h_i, ge_d_i, on =["periodo","grupo_etario"])
ge1_i = pd.merge(ge_i, ge_nv_i, on =["periodo","grupo_etario"])
ge1_i

,periodo,grupo_etario,%_vic_hechos,vic_hechos,%_des,des,%_nv,no_vic
0,2019,60 años o más,11.70,101163.00,12.50,87647.00,5.70,324570.00
1,2019,Entre 0 y 5 años,19.80,91959.00,19.40,88804.00,15.30,693852.00
2,2019,Entre 12 y 17 años,16.60,96631.00,16.20,92461.00,14.00,641638.00
3,2019,Entre 18 y 28 años,10.70,164991.00,10.90,160690.00,7.70,627535.00
4,2019,Entre 29 y 59 años,12.20,452132.00,12.60,418014.00,6.60,1100464.00
5,2019,Entre 6 y 11 años,19.00,134925.00,18.70,131278.00,15.00,721159.00
6,2020,60 años o más,12.40,101232.00,13.60,89386.00,8.30,488356.00
7,2020,Entre 0 y 5 años,25.30,100476.00,25.00,97932.00,21.40,966612.00
8,2020,Entre 12 y 17 años,25.50,227823.00,25.50,225555.00,19.50,812748.00
9,2020,Entre 18 y 28 años,18.10,322411.00,18.30,315330.00,13.70,1077724.00


### Territorial

#### Zona geográfica

In [19]:
### agrupar los datos para sacar tabla y gráfica
clase_h_i = df_vict.groupby(["periodo","clase"]).apply(
    lambda x: pd.Series({
        # vic tot ipm
        '%_vic_hechos': round((x['indigente'] * x['es_victima']* x['fex_c_viv']).sum() / (x['es_victima'] * x['fex_c_viv']).sum() * 100, 1),
        'vic_hechos': round((x['indigente'] * x['fex_c_viv']).sum(), 0)})
).reset_index()

clase_d_i = df_vict.groupby(["periodo","clase"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_des': round((x['indigente'] * x['es_desplazado'] * x['fex_c_viv']).sum() / (x['es_desplazado'] * x['fex_c_viv']).sum() * 100, 1),
        'des': round((x['indigente'] * x['es_desplazado'] *  x['fex_c_viv']).sum(), 0)
    })
).reset_index()


clase_nv_i = df_vict.groupby(["periodo","clase"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_nv': round((x['indigente'] *  x["no_victima"]*x['fex_c']).sum() / (x["no_victima"] * x["fex_c"]).sum() * 100, 1),
        'no_vic': round((x['indigente'] * x["no_victima"]* x["fex_c"]).sum(), 0)
    })
).reset_index()

clase_i = pd.merge(clase_h_i, clase_d_i, on =["periodo","clase"])
clase1_i = pd.merge(clase_i, clase_nv_i, on =["periodo","clase"])
clase1_i

,periodo,clase,%_vic_hechos,vic_hechos,%_des,des,%_nv,no_vic
0,2019,1,9.10,494680.00,9.50,468665.00,6.60,2293796.00
1,2019,2,22.30,547119.00,22.30,510228.00,18.80,1815422.00
2,2020,1,19.80,1033091.00,20.70,989084.00,13.60,4710867.00
3,2020,2,19.90,543737.00,20.10,516502.00,17.80,1618344.00
4,2021,1,16.50,852259.00,17.40,824898.00,9.70,3397975.00
5,2021,2,20.40,582542.00,20.60,559086.00,18.50,1677952.00
6,2022,1,18.20,944452.00,19.30,913676.00,10.10,3489841.00
7,2022,2,26.20,797819.00,26.10,745286.00,22.50,2079569.00


#### Departamento

In [20]:
### agrupar los datos para sacar tabla y gráfica
depto_h_i = df_vict.groupby(["periodo","depto", "nombre_dpto"]).apply(
    lambda x: pd.Series({
        # vic tot ipm
        '%_vic_hechos': round((x['indigente'] * x['es_victima']* x['fex_c_viv']).sum() / (x['es_victima'] * x['fex_c_viv']).sum() * 100, 1),
        'vic_hechos': round((x['indigente'] * x['fex_c_viv']).sum(), 0)})
).reset_index()

depto_d_i = df_vict.groupby(["periodo","depto", "nombre_dpto"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_des': round((x['indigente'] * x['es_desplazado'] * x['fex_c_viv']).sum() / (x['es_desplazado'] * x['fex_c_viv']).sum() * 100, 1),
        'des': round((x['indigente'] * x['es_desplazado'] *  x['fex_c_viv']).sum(), 0)
    })
).reset_index()


depto_nv_i = df_vict.groupby(["periodo","depto", "nombre_dpto"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_nv': round((x['indigente'] *  x["no_victima"]*x['fex_c']).sum() / (x["no_victima"] * x["fex_c"]).sum() * 100, 1),
        'no_vic': round((x['indigente'] * x["no_victima"]* x["fex_c"]).sum(), 0)
    })
).reset_index()

depto_i = pd.merge(depto_h_i, depto_d_i, on =["periodo","depto", "nombre_dpto"])
depto1_i = pd.merge(depto_i, depto_nv_i, on =["periodo","depto", "nombre_dpto"])
depto1_i

,periodo,depto,nombre_dpto,%_vic_hechos,vic_hechos,%_des,des,%_nv,no_vic
0,2019,05,Antioquia,8.40,113935.00,8.90,107524.00,6.10,321797.00
1,2019,08,Atlántico,3.20,6848.00,3.40,6848.00,3.50,88686.00
2,2019,11,Bogotá,6.20,32945.00,6.60,32219.00,4.10,326260.00
3,2019,13,Bolívar,15.60,62455.00,15.70,60615.00,10.60,213533.00
4,2019,15,Boyacá,9.60,5128.00,11.40,5086.00,7.10,101328.00
5,2019,17,Caldas,6.40,9393.00,6.40,8195.00,4.80,48453.00
6,2019,18,Caquetá,10.30,27512.00,10.70,26956.00,10.80,55883.00
7,2019,19,Cauca,26.60,88879.00,28.00,80670.00,23.80,304858.00
8,2019,20,Cesar,21.50,65274.00,22.10,63373.00,19.70,209645.00
9,2019,23,Córdoba,12.20,38637.00,12.60,37661.00,13.10,179537.00


#### Ciudades

In [21]:
### agrupar los datos para sacar tabla y gráfica
ciudad_h_i = df_vict.groupby(["periodo","cod_dane", "dominio"]).apply(
    lambda x: pd.Series({
        # vic tot ipm
        '%_vic_hechos': round((x['indigente'] * x['es_victima']* x['fex_c_viv']).sum() / (x['es_victima'] * x['fex_c_viv']).sum() * 100, 1),
        'vic_hechos': round((x['indigente'] * x['fex_c_viv']).sum(), 0)})
).reset_index()

ciudad_d_i = df_vict.groupby(["periodo","cod_dane", "dominio"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_des': round((x['indigente'] * x['es_desplazado'] * x['fex_c_viv']).sum() / (x['es_desplazado'] * x['fex_c_viv']).sum() * 100, 1),
        'des': round((x['indigente'] * x['es_desplazado'] *  x['fex_c_viv']).sum(), 0)
    })
).reset_index()


ciudad_nv_i = df_vict.groupby(["periodo","cod_dane", "dominio"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_nv': round((x['indigente'] *  x["no_victima"]*x['fex_c']).sum() / (x["no_victima"] * x["fex_c"]).sum() * 100, 1),
        'no_vic': round((x['indigente'] * x["no_victima"]* x["fex_c"]).sum(), 0)
    })
).reset_index()

ciudad_i = pd.merge(ciudad_h_i, ciudad_d_i, on =["periodo","cod_dane", "dominio"])
ciudad1_i = pd.merge(ciudad_i, ciudad_nv_i, on =["periodo","cod_dane", "dominio"])
ciudad1_i

,periodo,cod_dane,dominio,%_vic_hechos,vic_hechos,%_des,des,%_nv,no_vic
0,2019,05001,MEDELLIN,4.80,32561.00,5.20,29721.00,3.50,120405.00
1,2019,06001,CALI,7.00,19730.00,7.10,18211.00,4.60,109938.00
2,2019,08001,BARRANQUILLA,3.50,5360.00,3.70,5360.00,3.60,65942.00
3,2019,11001,BOGOTA,6.20,32945.00,6.60,32219.00,4.10,326260.00
4,2019,13001,CARTAGENA,3.10,1471.00,3.00,1355.00,3.00,29393.00
5,2019,15001,TUNJA,8.50,606.00,8.80,564.00,5.60,10640.00
6,2019,17001,MANIZALES,1.60,486.00,1.80,486.00,1.90,7869.00
7,2019,18001,FLORENCIA,8.00,5369.00,8.20,5125.00,7.70,9744.00
8,2019,19001,POPAYAN,13.90,6339.00,15.00,5954.00,11.10,25790.00
9,2019,20001,VALLEDUPAR,8.60,6077.00,8.70,5753.00,9.40,36299.00


### Exportar tablas

In [22]:
salida = p2 + "PM_PE_dpto_ciudades_19_22.xlsx"
writer = pd.ExcelWriter(salida)
#tablas

tablas = [sexo1, sexo1_i, ge1, ge1_i,
          depto1, ciudad1, depto1_i, ciudad1_i]

pestañas = ['sexo_PM', 'rango_edad_PM', 'sexo_PE', 'rango_edad_PE', 
            'p_mon_dpto', 'p_mon_ciudad','p_ext_dpto', 'p_ext_ciudad']
#exportar cada tabla como una pestaña del archivo excel
for i,j in zip(tablas, pestañas):
    i.to_excel(writer, sheet_name = j, header = True, index=True)

writer.close()

## Índice de pobreza multidimensional

In [10]:
# misma ruta del inciio del notebook
p2 = "C:/Users/ivan.corredor/pobreza_calculos_2019_2022/2023/multidimensional/salidas/"

In [11]:
## base de IPM nal 2019-2022
nal_19_22 = pd.read_csv(p2 + "nal_19_22.csv", sep=";", low_memory=False, decimal=",")

## base de IPM departamental 2019-2022
dpto_19_22 = pd.read_csv(p2 + "dpto_19_22.csv", sep=";", low_memory=False, decimal=",")

In [12]:
## borrar columnas innecesariosf
nal_19_22.drop(columns=["Unnamed: 0"], inplace =True)
dpto_19_22.drop(columns=["Unnamed: 0"], inplace =True)

# pasar a minuscular nombres de las variables
nal_19_22.columns = nal_19_22.columns.str.lower()
dpto_19_22.columns = dpto_19_22.columns.str.lower()

# asignar formato correcto
names = ['pobre', 'sexo', 'vic2', 'despla2', 'no_victima', 'fexhogvic', 'fex_c_h', 'fexp', "p3", "p6051"]
for i in names:
    nal_19_22[i] = pd.to_numeric(nal_19_22[i], errors='coerce')
    dpto_19_22[i] = pd.to_numeric(dpto_19_22[i], errors='coerce')

In [26]:
nal_19_22.head()

,periodo,directorio,secuencia_encuesta_pvh,secuencia_p_h,p5010,p8526,p8530,fex_c_h,region,personas,paredes,pisos,alcantarillado,acueducto,empleo_formal,desempleo_larga_duracion,barreras_acceso_salud,aseguramiento_salud,trabajo_infantil,atencion_integral,inasistencia_escolar,rezago_escolar,alfabetismo,logro_educativo,hacinamiento,ipm,pobre,fexp,secuencia_encuesta_p,secuencia_p_pv,orden,sexo,edad,fex_c_pv,p6051,p6090,p5665,p8563,p51,p55,p774,p6160,p8586,p8587,p8587s1,p1088,p1088s1,p6180,p6240,p6250,p6260,p6270,p6351,p6390,p7250,p6920,secuencia_encuesta_vic,es_victima,es_desplazado,fex_c_victi,fex_c_des,indicator_column,vic2,despla2,fexhogvic,indicator_column2,secuencia_encuesta_viv,secuencia_p,p3,p4005,p4015,p8520s3,p8520s5,indicator_column3,no_victima,grupo_etario,nombre_region,p1075,p1077s21,p1077s22,p1077s23,p1082s2
0,2019,7120001,1,1,2,1,1,208.35,1,4,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0.15,0,833.40,1,1,1,2,44,208.34913730011,1,1,1,1,,,,1,2,7,2,,,,4,1,,,,,,2,NaN,NaN,NaN,NaN,NaN,left_only,0,0,NaN,both,1,1,1,1,4,1,1,both,1,Entre 29 y 59 años,Caribe,NaN,NaN,NaN,NaN,NaN
1,2019,7120001,1,1,2,1,1,208.35,1,4,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0.15,0,833.40,2,1,2,1,48,208.34913730011,2,1,1,1,,,,1,2,11,5,,,,1,,,,,,,1,NaN,NaN,NaN,NaN,NaN,left_only,0,0,NaN,both,1,1,1,1,4,1,1,both,1,Entre 29 y 59 años,Caribe,NaN,NaN,NaN,NaN,NaN
2,2019,7120001,1,1,2,1,1,208.35,1,4,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0.15,0,833.40,3,1,3,1,13,208.34913730011,3,1,1,1,,,,1,1,,,3,7,2,3,2,2,2,,,,,NaN,NaN,NaN,NaN,NaN,left_only,0,0,NaN,both,1,1,1,1,4,1,1,both,1,Entre 12 y 17 años,Caribe,NaN,NaN,NaN,NaN,NaN
3,2019,7120001,1,1,2,1,1,208.35,1,4,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0.15,0,833.40,4,1,4,1,24,208.34913730011,14,1,2,,,,,1,1,,,8,1,2,1,,,,,,,1,NaN,NaN,NaN,NaN,NaN,left_only,0,0,NaN,both,1,1,1,1,4,1,1,both,1,Entre 18 y 28 años,Caribe,NaN,NaN,NaN,NaN,NaN
4,2019,7120002,1,1,4,1,1,306.48,1,10,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0.2,0,3064.77,1,1,1,1,61,306.477351890009,1,1,2,,,,,1,2,5,11,,,,1,,,,,,,1,NaN,NaN,NaN,NaN,NaN,left_only,1,1,438.85,both,1,1,1,1,4,1,1,both,0,60 años o más,Caribe,NaN,NaN,NaN,NaN,NaN


### Nacionales

In [27]:
ipm_h = nal_19_22.groupby(["periodo"]).apply(
    lambda x: pd.Series({
        # vic tot ipm
        '%_vic_hechos': round((x['pobre'] * x['vic2']* x['fexhogvic']).sum() / (x['vic2'] * x['fexhogvic']).sum() * 100, 1),
        'vic_hechos': round((x['pobre'] * x['fexhogvic']).sum(), 0)})
).reset_index()

ipm_d = nal_19_22.groupby(["periodo"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_des': round((x['pobre'] * x['despla2'] * x['fexhogvic']).sum() / (x['despla2'] * x['fexhogvic']).sum() * 100, 1),
        'des': round((x['pobre'] * x['despla2'] *  x['fexhogvic']).sum(), 0)
    })
).reset_index()

ipm_nal = nal_19_22.groupby(["periodo"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_nal': round((x['pobre'] * x['fex_c_h']).sum() / (x["fex_c_h"]).sum() * 100, 1),
        'nal': round((x['pobre'] * x["fex_c_h"]).sum(), 0)
    })
).reset_index()

ipm_nv = nal_19_22.groupby(["periodo"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_nv': round((x['pobre'] * x["no_victima"] * x['fex_c_h']).sum() / (x["no_victima"] * x['fex_c_h']).sum() * 100, 1),
        'no_vic': round((x['pobre'] * x['no_victima']* x['fex_c_h']).sum(), 0)
    })
).reset_index()

from functools import reduce
# List of DataFrames to merge
frames_s = [ipm_h, ipm_d, ipm_nal, ipm_nv]

# Use reduce to iteratively merge the DataFrames
ipm = reduce(lambda left, right: pd.merge(left, right, on=["periodo"]), frames_s)
ipm

,periodo,%_vic_hechos,vic_hechos,%_des,des,%_nal,nal,%_nv,no_vic
0,2019,26.70,3826068.00,27.60,3596910.00,17.50,8560324.00,14.60,5507851.00
1,2020,28.90,4190793.00,30.30,3995066.00,18.10,9049099.00,14.80,5674923.00
2,2021,24.40,3394359.00,25.30,3190298.00,16.00,8078146.00,13.50,5261926.00
3,2022,20.50,3053665.00,21.50,2898534.00,12.90,6607135.00,10.60,4173863.00


### Sexo

In [27]:
### agrupar los datos para sacar tabla y gráfica
### cifras nacionales quitando la variable P3
sexipm_h = nal_19_22[nal_19_22["p6051"]==1].groupby(["periodo", "sexo"]).apply(
    lambda x: pd.Series({
        # vic tot ipm
        '%_vic_hechos': round((x['pobre'] * x['vic2']* x['fexhogvic']).sum() / (x['vic2'] * x['fexhogvic']).sum() * 100, 1),
        'vic_hechos': round((x['pobre'] * x['fexhogvic']).sum(), 0)})
).reset_index()

sexipm_d = nal_19_22[nal_19_22["p6051"]==1].groupby(["periodo", "sexo"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_des': round((x['pobre'] * x['despla2'] * x['fexhogvic']).sum() / (x['despla2'] * x['fexhogvic']).sum() * 100, 1),
        'des': round((x['pobre'] * x['despla2'] *  x['fexhogvic']).sum(), 0)
    })
).reset_index()

sexipm_nal = nal_19_22[nal_19_22["p6051"]==1].groupby(["periodo", "sexo"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_nal': round((x['pobre'] * x['fexp']).sum() / (x["fexp"]).sum() * 100, 1),
        'nal': round((x['pobre'] * x["fexp"]).sum(), 0)
    })
).reset_index()

sexipm_nv = nal_19_22[nal_19_22["p6051"]==1].groupby(["periodo", "sexo"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_nv': round((x['pobre'] * x["no_victima"] * x['fexp']).sum() / (x["no_victima"] * x['fexp']).sum() * 100, 1),
        'no_vic': round((x['pobre'] * x['no_victima']* x['fexp']).sum(), 0)
    })
).reset_index()

from functools import reduce
# List of DataFrames to merge
frames_s = [sexipm_h, sexipm_d, sexipm_nal, sexipm_nv]

# Use reduce to iteratively merge the DataFrames
sexipm = reduce(lambda left, right: pd.merge(left, right, on=["periodo", "sexo"]), frames_s)
sexipm

,periodo,sexo,%_vic_hechos,vic_hechos,%_des,des,%_nal,nal,%_nv,no_vic
0,2019,1,21.30,503663.00,22.00,472334.00,16.60,5135093.00,14.10,3391328.00
1,2019,2,24.70,382325.00,25.70,355706.00,18.90,3425231.00,15.50,2116523.00
2,2020,1,23.20,551919.00,24.20,520438.00,17.20,5292950.00,14.30,3400626.00
3,2020,2,27.50,453555.00,29.20,430079.00,19.60,3756148.00,15.70,2274297.00
4,2021,1,20.80,462250.00,21.80,437219.00,14.70,4294284.00,12.20,2778605.00
5,2021,2,21.70,394424.00,22.50,363983.00,17.60,3783862.00,15.20,2483322.00
6,2022,1,16.90,403999.00,17.70,380402.00,12.00,3462854.00,9.80,2182727.00
7,2022,2,18.60,375366.00,19.40,347579.00,14.10,3144281.00,11.70,1991135.00


In [28]:
nal_19_22.head()

,periodo,directorio,secuencia_encuesta_pvh,secuencia_p_h,p5010,p8526,p8530,fex_c_h,region,personas,paredes,pisos,alcantarillado,acueducto,empleo_formal,desempleo_larga_duracion,barreras_acceso_salud,aseguramiento_salud,trabajo_infantil,atencion_integral,inasistencia_escolar,rezago_escolar,alfabetismo,logro_educativo,hacinamiento,ipm,pobre,fexp,secuencia_encuesta_p,secuencia_p_pv,orden,sexo,edad,fex_c_pv,p6051,p6090,p5665,p8563,p51,p55,p774,p6160,p8586,p8587,p8587s1,p1088,p1088s1,p6180,p6240,p6250,p6260,p6270,p6351,p6390,p7250,p6920,secuencia_encuesta_vic,es_victima,es_desplazado,fex_c_victi,fex_c_des,indicator_column,vic2,despla2,fexhogvic,indicator_column2,secuencia_encuesta_viv,secuencia_p,p3,p4005,p4015,p8520s3,p8520s5,indicator_column3,no_victima,grupo_etario,nombre_region,p1075,p1077s21,p1077s22,p1077s23,p1082s2
0,2019,7120001,1,1,2,1,1,208.35,1,4,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0.15,0,833.40,1,1,1,2,44,208.34913730011,1,1,1,1,,,,1,2,7,2,,,,4,1,,,,,,2,NaN,NaN,NaN,NaN,NaN,left_only,0,0,NaN,both,1,1,1,1,4,1,1,both,1,Entre 29 y 59 años,Caribe,NaN,NaN,NaN,NaN,NaN
1,2019,7120001,1,1,2,1,1,208.35,1,4,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0.15,0,833.40,2,1,2,1,48,208.34913730011,2,1,1,1,,,,1,2,11,5,,,,1,,,,,,,1,NaN,NaN,NaN,NaN,NaN,left_only,0,0,NaN,both,1,1,1,1,4,1,1,both,1,Entre 29 y 59 años,Caribe,NaN,NaN,NaN,NaN,NaN
2,2019,7120001,1,1,2,1,1,208.35,1,4,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0.15,0,833.40,3,1,3,1,13,208.34913730011,3,1,1,1,,,,1,1,,,3,7,2,3,2,2,2,,,,,NaN,NaN,NaN,NaN,NaN,left_only,0,0,NaN,both,1,1,1,1,4,1,1,both,1,Entre 12 y 17 años,Caribe,NaN,NaN,NaN,NaN,NaN
3,2019,7120001,1,1,2,1,1,208.35,1,4,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0.15,0,833.40,4,1,4,1,24,208.34913730011,14,1,2,,,,,1,1,,,8,1,2,1,,,,,,,1,NaN,NaN,NaN,NaN,NaN,left_only,0,0,NaN,both,1,1,1,1,4,1,1,both,1,Entre 18 y 28 años,Caribe,NaN,NaN,NaN,NaN,NaN
4,2019,7120002,1,1,4,1,1,306.48,1,10,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0.2,0,3064.77,1,1,1,1,61,306.477351890009,1,1,2,,,,,1,2,5,11,,,,1,,,,,,,1,NaN,NaN,NaN,NaN,NaN,left_only,1,1,438.85,both,1,1,1,1,4,1,1,both,0,60 años o más,Caribe,NaN,NaN,NaN,NaN,NaN


### Rango Etario

In [29]:
### agrupar los datos para sacar tabla y gráfica
eta_h = nal_19_22[nal_19_22["p6051"]==1].groupby(["periodo", "grupo_etario"]).apply(
    lambda x: pd.Series({
        # vic tot ipm
        '%_vic_hechos': round((x['pobre'] * x['vic2']* x['fexhogvic']).sum() / (x['vic2'] * x['fexhogvic']).sum() * 100, 1),
        'vic_hechos': round((x['pobre'] * x['fexhogvic']).sum(), 0)})
).reset_index()

eta_d = nal_19_22[nal_19_22["p6051"]==1].groupby(["periodo", "grupo_etario"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_des': round((x['pobre'] * x['despla2'] * x['fexhogvic']).sum() / (x['despla2'] * x['fexhogvic']).sum() * 100, 1),
        'des': round((x['pobre'] * x['despla2'] *  x['fexhogvic']).sum(), 0)
    })
).reset_index()

eta_nal = nal_19_22[nal_19_22["p6051"]==1].groupby(["periodo", "grupo_etario"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_nal': round((x['pobre'] * x['fexp']).sum() / (x["fexp"]).sum() * 100, 1),
        'nal': round((x['pobre'] * x["fexp"]).sum(), 0)
    })
).reset_index()

eta_nv = nal_19_22[nal_19_22["p6051"]==1].groupby(["periodo", "grupo_etario"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_nv': round((x['pobre'] * x["no_victima"] * x['fexp']).sum() / (x["no_victima"] * x["fexp"]).sum() * 100, 1),
        'no_vic': round((x['pobre'] * x["no_victima"]* x["fexp"]).sum(), 0)
    })
).reset_index()

from functools import reduce
# List of DataFrames to merge
frames_e = [eta_h, eta_d, eta_nal, eta_nv]

# Use reduce to iteratively merge the DataFrames
eta = reduce(lambda left, right: pd.merge(left, right, on=["periodo", "grupo_etario"]), frames_e)
eta

,periodo,grupo_etario,%_vic_hechos,vic_hechos,%_des,des,%_nal,nal,%_nv,no_vic
0,2019,60 años o más,33.40,235324.00,35.50,212180.00,20.40,2279766.00,17.20,1582841.00
1,2019,Entre 12 y 17 años,61.60,1869.00,62.30,1869.00,57.00,19792.00,59.30,16178.00
2,2019,Entre 18 y 28 años,13.40,79694.00,13.60,78336.00,15.20,727935.00,14.80,510356.00
3,2019,Entre 29 y 59 años,21.80,569100.00,22.70,535656.00,16.70,5532831.00,13.60,3398476.00
4,2020,60 años o más,34.40,276956.00,38.00,252283.00,20.40,2278569.00,16.80,1532102.00
5,2020,Entre 12 y 17 años,62.80,2658.00,62.80,2658.00,57.20,15714.00,55.00,10698.00
6,2020,Entre 18 y 28 años,16.30,93049.00,16.40,91335.00,16.80,842556.00,15.60,556875.00
7,2020,Entre 29 y 59 años,23.90,632812.00,25.20,604241.00,17.50,5912259.00,14.00,3575247.00
8,2021,60 años o más,32.60,258497.00,36.00,234671.00,18.90,2153650.00,15.70,1472308.00
9,2021,Entre 12 y 17 años,57.10,3835.00,57.10,3835.00,61.50,15003.00,58.50,6937.00


In [22]:
nal_19_22[(nal_19_22["p6051"]==1) & (nal_19_22["edad"]<18)].groupby(["periodo", "edad"]).aggregate({"edad":np.size})

edad
periodo edad      
2019    13       1
        14       7
        15      15
        16      37
        17      85
2020    13       1
        14       2
        15      10
        16      21
        17      53
2021    12       1
        14       2
        15      12
        16      20
        17      63
2022    14       3
        15       9
        16      28
        17      46

### Zona geográfica

In [30]:
### agrupar los datos para sacar tabla y gráfica
### cifras nacionales quitando la variable P3
zona_h = nal_19_22.groupby(["periodo", "p3"]).apply(
    lambda x: pd.Series({
        # vic tot ipm
        '%_vic_hechos': round((x['pobre'] * x['vic2']* x['fexhogvic']).sum() / (x['vic2'] * x['fexhogvic']).sum() * 100, 1),
        'vic_hechos': round((x['pobre'] * x['fexhogvic']).sum(), 0)})
).reset_index()

zona_d = nal_19_22.groupby(["periodo", "p3"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_des': round((x['pobre'] * x['despla2'] * x['fexhogvic']).sum() / (x['despla2'] * x['fexhogvic']).sum() * 100, 1),
        'des': round((x['pobre'] * x['despla2'] *  x['fexhogvic']).sum(), 0)
    })
).reset_index()

zona_nal = nal_19_22.groupby(["periodo", "p3"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_nal': round((x['pobre'] * x['fex_c_h']).sum() / (x['fex_c_h']).sum() * 100, 1),
        'nal': round((x['pobre'] * x['fex_c_h']).sum(), 0)
    })
).reset_index()

zona_nv = nal_19_22.groupby(["periodo", "p3"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_nv': round((x['pobre'] * x['no_victima'] * x['fex_c_h']).sum() / (x['no_victima'] * x['fex_c_h']).sum() * 100, 1),
        'no_vic': round((x['pobre'] * x['no_victima']* x['fex_c_h']).sum(), 0)
    })
).reset_index()

from functools import reduce
# List of DataFrames to merge
frames = [zona_h, zona_d, zona_nal, zona_nv]

# Use reduce to iteratively merge the DataFrames
zona_r = reduce(lambda left, right: pd.merge(left, right, on=["periodo", "p3"]), frames)
zona_r

,periodo,p3,%_vic_hechos,vic_hechos,%_des,des,%_nal,nal,%_nv,no_vic
0,2019,1,21.30,2095771.00,22.30,1976945.00,12.30,4635453.00,10.20,3061830.00
1,2019,2,35.50,444904.00,35.80,419870.00,29.50,1059467.00,26.40,609970.00
2,2019,3,40.20,1285394.00,40.30,1200095.00,36.90,2865404.00,34.50,1836051.00
3,2020,1,21.00,2027037.00,22.30,1932852.00,12.50,4799635.00,10.30,3167586.00
4,2020,2,36.50,634549.00,37.20,613954.00,30.10,1099802.00,26.20,588902.00
5,2020,3,49.40,1529207.00,50.20,1448260.00,40.30,3149661.00,36.10,1918435.00
6,2021,1,18.00,1637049.00,18.70,1512206.00,11.50,4494489.00,9.90,3136268.00
7,2021,2,31.00,521894.00,31.10,498095.00,26.20,960273.00,23.30,527270.00
8,2021,3,39.40,1235416.00,40.30,1179997.00,33.40,2623384.00,30.40,1598388.00
9,2022,1,14.70,1422301.00,15.60,1343792.00,8.70,3445498.00,7.30,2312782.00


### Región

In [31]:
# ipm por regiones. nacional
### agrupar los datos para sacar tabla y gráfica
reg_h = nal_19_22.groupby(["periodo", "region", "nombre_region"]).apply(
    lambda x: pd.Series({
        # vic tot ipm
        '%_vic_hechos': round((x['pobre'] * x['vic2']* x['fexhogvic']).sum() / (x['vic2'] * x['fexhogvic']).sum() * 100, 1),
        'vic_hechos': round((x['pobre'] * x['fexhogvic']).sum(), 0)})
).reset_index()

reg_d = nal_19_22.groupby(["periodo", "region", "nombre_region"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_des': round((x['pobre'] * x['despla2'] * x['fexhogvic']).sum() / (x['despla2'] * x['fexhogvic']).sum() * 100, 1),
        'des': round((x['pobre'] * x['despla2'] *  x['fexhogvic']).sum(), 0)
    })
).reset_index()

reg_nal = nal_19_22.groupby(["periodo", "region", "nombre_region"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_nal': round((x['pobre'] * x['fex_c_h']).sum() / (x['fex_c_h']).sum() * 100, 1),
        'nal': round((x['pobre'] * x['fex_c_h']).sum(), 0)
    })
).reset_index()

reg_nv = nal_19_22.groupby(["periodo", "region", "nombre_region"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_nv': round((x['pobre'] * x["no_victima"] * x['fex_c_h']).sum() / (x["no_victima"] * x['fex_c_h']).sum() * 100, 1),
        'no_vic': round((x['pobre'] * x["no_victima"]* x['fex_c_h']).sum(), 0)
    })
).reset_index()

from functools import reduce
# List of DataFrames to merge
frames_r = [reg_h, reg_d, reg_nal, reg_nv]

# Use reduce to iteratively merge the DataFrames
reg = reduce(lambda left, right: pd.merge(left, right, on=["periodo", "region", "nombre_region"]), frames_r)
reg

,periodo,region,nombre_region,%_vic_hechos,vic_hechos,%_des,des,%_nal,nal,%_nv,no_vic
0,2019,1,Caribe,33.70,1313313.00,34.10,1267425.00,28.10,3141286.00,26.20,2071384.00
1,2019,2,Oriental,22.20,389681.00,21.80,342243.00,15.20,1413379.00,13.90,1085014.00
2,2019,3,Central,23.60,405111.00,23.80,365638.00,15.20,817723.00,13.10,544527.00
3,2019,4,Pacífica (sin valle),35.00,722084.00,36.00,703347.00,26.30,961516.00,21.50,466747.00
4,2019,5,Bogotá,10.50,60006.00,9.10,46165.00,7.10,536933.00,6.90,492710.00
5,2019,6,Antioquia,22.10,596548.00,24.10,547892.00,15.70,1034423.00,10.50,450724.00
6,2019,7,Valle del Cauca,20.10,244422.00,21.10,233386.00,10.80,487212.00,8.20,296148.00
7,2019,8,San Andrés y Providencia,0.00,0.00,0.00,0.00,8.20,3414.00,8.40,3414.00
8,2019,9,Orinoquía y Amazonía,24.50,94903.00,25.60,90813.00,20.20,164438.00,18.80,97182.00
9,2020,1,Caribe,34.10,1437718.00,34.40,1385198.00,28.70,3287161.00,26.50,2129597.00


### Región y Sexo

In [32]:
# ipm por regiones. nacional
### agrupar los datos para sacar tabla y gráfica
reg_s_h = nal_19_22[nal_19_22["p6051"]==1].groupby(["periodo", "region", "nombre_region","sexo"]).apply(
    lambda x: pd.Series({
        # vic tot ipm
        '%_vic_hechos': round((x['pobre'] * x['vic2']* x['fexhogvic']).sum() / (x['vic2'] * x['fexhogvic']).sum() * 100, 1),
        'vic_hechos': round((x['pobre'] * x['fexhogvic']).sum(), 0)})
).reset_index()

reg_s_d = nal_19_22[nal_19_22["p6051"]==1].groupby(["periodo", "region", "nombre_region","sexo"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_des': round((x['pobre'] * x['despla2'] * x['fexhogvic']).sum() / (x['despla2'] * x['fexhogvic']).sum() * 100, 1),
        'des': round((x['pobre'] * x['despla2'] *  x['fexhogvic']).sum(), 0)
    })
).reset_index()

reg_s_nal = nal_19_22[nal_19_22["p6051"]==1].groupby(["periodo", "region", "nombre_region","sexo"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_nal': round((x['pobre'] * x['fexp']).sum() / (x['fexp']).sum() * 100, 1),
        'nal': round((x['pobre'] * x['fexp']).sum(), 0)
    })
).reset_index()

reg_s_nv = nal_19_22[nal_19_22["p6051"]==1].groupby(["periodo", "region", "nombre_region","sexo"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_nv': round((x['pobre'] * x["no_victima"] * x['fexp']).sum() / (x["no_victima"] * x['fexp']).sum() * 100, 1),
        'no_vic': round((x['pobre'] * x["no_victima"]* x['fexp']).sum(), 0)
    })
).reset_index()

from functools import reduce
# List of DataFrames to merge
frames_rs = [reg_s_h, reg_s_d, reg_s_nal, reg_s_nv]

# Use reduce to iteratively merge the DataFrames
reg_s = reduce(lambda left, right: pd.merge(left, right, on=["periodo", "region", "nombre_region", "sexo"]), frames_rs)
reg_s

,periodo,region,nombre_region,sexo,%_vic_hechos,vic_hechos,%_des,des,%_nal,nal,%_nv,no_vic
0,2019,1,Caribe,1,29.10,173033.00,29.50,168539.00,27.80,1931850.00,26.10,1302253.00
1,2019,1,Caribe,2,30.90,118962.00,31.20,113093.00,28.80,1209436.00,26.20,769132.00
2,2019,2,Oriental,1,17.00,52732.00,16.40,44982.00,13.90,845136.00,12.80,657024.00
3,2019,2,Oriental,2,22.40,39266.00,23.30,36452.00,17.80,568243.00,16.20,427990.00
4,2019,3,Central,1,19.40,59632.00,19.30,53301.00,14.60,505085.00,12.60,336820.00
5,2019,3,Central,2,20.90,37038.00,21.60,33094.00,16.30,312638.00,13.90,207707.00
6,2019,4,Pacífica (sin valle),1,28.60,101270.00,29.60,98064.00,25.00,614619.00,20.20,308312.00
7,2019,4,Pacífica (sin valle),2,32.50,75107.00,33.10,72395.00,29.00,346897.00,24.40,158434.00
8,2019,5,Bogotá,1,7.30,8976.00,5.70,6420.00,6.90,338594.00,6.80,313110.00
9,2019,5,Bogotá,2,14.60,8380.00,12.20,5681.00,7.20,198339.00,6.90,179599.00


### Departamento

In [33]:
# base departamentos
### agrupar los datos para sacar tabla y gráfica
dpto_h = dpto_19_22.groupby(["periodo", "departamento", "nombre_departamento"]).apply(
    lambda x: pd.Series({
        # vic tot ipm
        '%_vic_hechos': round((x['pobre'] * x['vic2']* x['fexhogvic']).sum() / (x['vic2'] * x['fexhogvic']).sum() * 100, 1),
        'vic_hechos': round((x['pobre'] * x['fexhogvic']).sum(), 0)})
).reset_index()

dpto_d = dpto_19_22.groupby(["periodo", "departamento", "nombre_departamento"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_des': round((x['pobre'] * x['despla2'] * x['fexhogvic']).sum() / (x['despla2'] * x['fexhogvic']).sum() * 100, 1),
        'des': round((x['pobre'] * x['despla2'] *  x['fexhogvic']).sum(), 0)
    })
).reset_index()

dpto_nal = dpto_19_22.groupby(["periodo", "departamento", "nombre_departamento"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_nal': round((x['pobre'] * x['fex_c_h']).sum() / (x['fex_c_h']).sum() * 100, 1),
        'nal': round((x['pobre'] * x['fex_c_h']).sum(), 0)
    })
).reset_index()

dpto_nv = dpto_19_22.groupby(["periodo", "departamento", "nombre_departamento"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_nv': round((x['pobre'] * x["no_victima"] * x['fex_c_h']).sum() / (x["no_victima"] * x['fex_c_h']).sum() * 100, 1),
        'no_vic': round((x['pobre'] * x["no_victima"]* x['fex_c_h']).sum(), 0)
    })
).reset_index()

from functools import reduce
# List of DataFrames to merge
frames_d = [dpto_h, dpto_d, dpto_nal, dpto_nv]

# Use reduce to iteratively merge the DataFrames
dpto = reduce(lambda left, right: pd.merge(left, right, on=["periodo", "departamento", "nombre_departamento"]), frames_d)
dpto

,periodo,departamento,nombre_departamento,%_vic_hechos,vic_hechos,%_des,des,%_nal,nal,%_nv,no_vic
0,2019,5,Antioquia,22.10,596548.00,24.10,547892.00,15.70,1034423.00,10.50,450724.00
1,2019,8,Atlántico,17.40,77084.00,17.40,72077.00,14.90,398148.00,14.00,318476.00
2,2019,11,Bogotá,10.60,60457.00,9.10,46545.00,7.10,541410.00,6.90,496682.00
3,2019,13,Bolívar,37.20,324998.00,37.40,321385.00,26.90,577248.00,22.50,329279.00
4,2019,15,Boyacá,12.60,13451.00,12.30,11144.00,12.80,157691.00,12.80,146311.00
5,2019,17,Caldas,23.70,64086.00,26.20,59665.00,14.30,144258.00,12.50,104048.00
6,2019,18,Caquetá,29.30,109711.00,29.00,102804.00,25.70,104635.00,22.30,36868.00
7,2019,19,Cauca,32.60,211885.00,34.40,207138.00,24.00,355249.00,23.10,239568.00
8,2019,20,Cesar,29.40,160035.00,30.00,155327.00,25.50,322344.00,24.10,180352.00
9,2019,23,Córdoba,40.60,282414.00,41.10,266619.00,34.70,629625.00,32.50,415958.00


### Departamento y sexo

In [34]:
# ipm por regiones. nacional
### agrupar los datos para sacar tabla y gráfica
dpto_s_h = dpto_19_22[dpto_19_22["p6051"]==1].groupby(["periodo", "departamento", "nombre_departamento", "sexo"]).apply(
    lambda x: pd.Series({
        # vic tot ipm
        '%_vic_hechos': round((x['pobre'] * x['vic2']* x['fexhogvic']).sum() / (x['vic2'] * x['fexhogvic']).sum() * 100, 1),
        'vic_hechos': round((x['pobre'] * x['fexhogvic']).sum(), 0)})
).reset_index()

dpto_s_d = dpto_19_22[dpto_19_22["p6051"]==1].groupby(["periodo","departamento", "nombre_departamento", "sexo"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_des': round((x['pobre'] * x['despla2'] * x['fexhogvic']).sum() / (x['despla2'] * x['fexhogvic']).sum() * 100, 1),
        'des': round((x['pobre'] * x['despla2'] *  x['fexhogvic']).sum(), 0)
    })
).reset_index()

dpto_s_nal = dpto_19_22[dpto_19_22["p6051"]==1].groupby(["periodo", "departamento", "nombre_departamento", "sexo"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_nal': round((x['pobre'] * x['fexp']).sum() / (x['fexp']).sum() * 100, 1),
        'nal': round((x['pobre'] * x['fexp']).sum(), 0)
    })
).reset_index()

dpto_s_nv = dpto_19_22[dpto_19_22["p6051"]==1].groupby(["periodo", "departamento", "nombre_departamento", "sexo"]).apply(
    lambda x: pd.Series({
        # vic des ipm
        '%_nv': round((x['pobre'] * x["no_victima"] * x['fexp']).sum() / (x["no_victima"] * x['fexp']).sum() * 100, 1),
        'no_vic': round((x['pobre'] * x["no_victima"]* x['fexp']).sum(), 0)
    })
).reset_index()

from functools import reduce
# List of DataFrames to merge
frames_ds = [dpto_s_h, dpto_s_d, dpto_s_nal, dpto_s_nv]

# Use reduce to iteratively merge the DataFrames
dpto_s = reduce(lambda left, right: pd.merge(left, right, 
                                             on=["periodo", "departamento", "nombre_departamento", "sexo"]), frames_ds)
dpto_s

,periodo,departamento,nombre_departamento,sexo,%_vic_hechos,vic_hechos,%_des,des,%_nal,nal,%_nv,no_vic
0,2019,5,Antioquia,1,17.20,71124.00,19.10,65619.00,15.30,562428.00,10.80,265742.00
1,2019,5,Antioquia,2,19.50,64687.00,21.00,58037.00,16.20,471994.00,10.00,184982.00
2,2019,8,Atlántico,1,12.60,7783.00,12.60,7264.00,14.60,242732.00,13.80,195158.00
3,2019,8,Atlántico,2,17.90,6226.00,18.30,5778.00,15.50,155416.00,14.30,123318.00
4,2019,11,Bogotá,1,7.30,9035.00,5.70,6469.00,7.00,341461.00,6.90,315705.00
5,2019,11,Bogotá,2,14.60,8411.00,12.20,5703.00,7.30,199950.00,6.90,180977.00
6,2019,13,Bolívar,1,37.30,46153.00,37.40,45220.00,26.90,338960.00,22.20,193647.00
7,2019,13,Bolívar,2,30.40,30184.00,30.70,29778.00,27.00,238288.00,23.10,135632.00
8,2019,15,Boyacá,1,12.30,2289.00,14.10,2175.00,12.60,106886.00,12.70,100364.00
9,2019,15,Boyacá,2,11.30,1369.00,9.70,975.00,13.20,50805.00,13.10,45947.00


### Exportar tablas IPM

In [35]:
salida2 = p3 + "ipm_19_22.xlsx"
writer2 = pd.ExcelWriter(salida2)

#tablas
tablas2 = [ipm, sexipm, eta, zona_r, reg, reg_s, dpto, dpto_s]
pestañas2 = ['general','sexo', 'rango_edad','zona', 'region', 'region y sexo', 'departamento', 'depto y sexo']

#exportar cada tabla como una pestaña del archivo excel
for i,j in zip(tablas2, pestañas2):
    i.to_excel(writer2, sheet_name = j, header = True, index=True)

writer2.close()

In [36]:
# # Scatter plot
# Año = [2019, 2020, 2021, 2022]
# Pobreza_nv = [p_nv_1, p_nv_2, p_nv_3, p_nv_4] 
# Pobreza_v = [p_v_1, p_v_2, p_v_3,  p_v_4]
# Dif =np.array(Pobreza_v) - np.array(Pobreza_nv)
# vectorize_round = np.vectorize(round)
# Dif = vectorize_round(Dif,2)

# plt.plot(Año, Pobreza_nv, label="No Víctima", marker = '.', color = "green")
# plt.plot(Año, Pobreza_v, label="Víctima Despl.", marker = '.', color = "orange")
# plt.plot(Año, Dif, label="Diferencia",ls = '-.', marker = '^', color = "blue")
# plt.legend(loc = "upper left")

# # Previous customizations
# plt.ylim(5,70)
# plt.xlabel('Año')
# plt.ylabel('[%]')
# plt.title("Tasa de pobreza Monetaria. Víctimas Desplazamiento vs No Víctimas\n" + "2019-2022", 
#     color = "Black")
# #plt.xticks([1000,10000,100000], ['1k','10k','100k'])
# # Additional customizations
# #plt.text(1550, 71, 'India')
# #plt.text(5700, 80, 'China')
# plt.xticks([2019, 2020, 2021, 2022], ["2019", "2020", "2021", "2022"] )
# # Add grid() call
# #plt.grid(True) #aggregate lines
# #etiquetar datos
# for x, y in zip(Año, Pobreza_nv):
#     plt.text(x, y, y,  ha='center', va='bottom', fontsize=10.5)
# for x, y in zip(Año, Pobreza_v):
#     plt.text(x, y, y,  ha='center', va='bottom', fontsize=10.5)
# for x, y in zip(Año, Dif):
#     plt.text(x, y, y,  ha='center', va='bottom', fontsize=10.5)
# # Show the plot
# plt.show()

NameError: name 'p_nv_1' is not defined